In [11]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("yemoonsaBigdata/datasets/Part3/504_x_test.csv")
X_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/504_x_train.csv")
y_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/504_y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'index': X_test.index, 'y_pred': pred}).to_csv('003000000.csv', index=False)

# 1.데이터 전처리

In [12]:
#결측치 확인
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7468 non-null   int64  
 1   model         7468 non-null   object 
 2   year          7468 non-null   int64  
 3   transmission  7468 non-null   object 
 4   mileage       7468 non-null   int64  
 5   fuelType      7468 non-null   object 
 6   tax           7468 non-null   int64  
 7   mpg           7468 non-null   float64
 8   engineSize    7468 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 525.2+ KB
None


In [13]:
#이상치 확인
print(X_train.describe())

                id         year        mileage          tax          mpg  \
count  7468.000000  7468.000000    7468.000000  7468.000000  7468.000000   
mean   3734.500000  2017.084762   24978.337306   125.513524    50.980704   
std    2155.970238     2.143985   23408.243740    66.791388    12.781446   
min       1.000000  1998.000000       1.000000     0.000000    19.300000   
25%    1867.750000  2016.000000    6000.000000   125.000000    40.900000   
50%    3734.500000  2017.000000   19276.500000   145.000000    50.400000   
75%    5601.250000  2019.000000   36722.750000   145.000000    58.900000   
max    7468.000000  2020.000000  168017.000000   580.000000   188.300000   

        engineSize  
count  7468.000000  
mean      1.923741  
std       0.594344  
min       0.000000  
25%       1.500000  
50%       2.000000  
75%       2.000000  
max       6.300000  


In [14]:
#결측치제거
y_train = y_train[(X_train['engineSize']!=0)]
X_train = X_train[(X_train['engineSize']!=0)]

In [15]:
#변수 설정
COL_DEL = ['id']
COL_NUM = ['year','mileage','tax','mpg','engineSize']
COL_CAT = ['model', 'transmission', 'fuelType']
COL_Y = ['price'] 

In [16]:
#학습테스트 데이터 분리
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train[COL_NUM+COL_CAT],
                                            y_train[COL_Y].values.ravel(),
                                            test_size=0.3)

In [17]:
#정규화
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])

X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [18]:
#범주화 
from sklearn.preprocessing import LabelEncoder

X=pd.concat([X_train[COL_CAT],X_test[COL_CAT]])

for col in COL_CAT:
    le = LabelEncoder()
    le.fit(X[col])

    X_tr[col] = le.transform(X_tr[col])
    X_val[col] = le.transform(X_val[col])
    X_test[col] = le.transform(X_test[col])

    print(col)
    print(le.classes_)

model
[' A1' ' A2' ' A3' ' A4' ' A5' ' A6' ' A7' ' A8' ' Q2' ' Q3' ' Q5' ' Q7'
 ' Q8' ' R8' ' RS3' ' RS4' ' RS5' ' RS6' ' RS7' ' S3' ' S4' ' S5' ' S8'
 ' SQ5' ' SQ7' ' TT']
transmission
['Automatic' 'Manual' 'Semi-Auto']
fuelType
['Diesel' 'Hybrid' 'Petrol']


# 2. 모형생성

In [30]:
#랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
modelRF = RandomForestRegressor(random_state=123)
modelRF.fit(X_tr,y_tr)

#부스팅
from xgboost import XGBRegressor
modelXGB = XGBRegressor(random_state=123)
modelXGB.fit(X_tr,y_tr)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=123, ...)

In [31]:
#예측값 생성
y_val_pred_RF = modelRF.predict(X_val)
y_val_pred_XGB = modelXGB.predict(X_val)

In [32]:
#예측값 비교 
import numpy as np 
from sklearn.metrics import mean_squared_error
scoreRF = np.sqrt(mean_squared_error(y_val, y_val_pred_RF))
scoreXGB = np.sqrt(mean_squared_error(y_val, y_val_pred_XGB))

print(scoreRF,scoreXGB)
#부스팅 채용 

3414.30618654044 3350.096971151789


In [37]:
#하이퍼파라미터 조정
modelXGB2 = XGBRegressor(n_estimators = 50, max_depth = 5, min_child_weight = 2, random_state=123)
modelXGB2.fit(X_tr,y_tr)

modelXGB3 = XGBRegressor(n_estimators = 50, max_depth = 10, min_child_weight = 2, random_state=123)
modelXGB3.fit(X_tr,y_tr)

modelXGB4 = XGBRegressor(n_estimators = 100, max_depth = 5, min_child_weight = 1, random_state=123)
modelXGB4.fit(X_tr,y_tr)

modelXGB5 = XGBRegressor(n_estimators = 100, max_depth = 10, min_child_weight = 2, random_state=123)
modelXGB5.fit(X_tr,y_tr)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=123, ...)

In [41]:
print(np.sqrt(mean_squared_error(y_val, modelXGB2.predict(X_val))))
print(np.sqrt(mean_squared_error(y_val, modelXGB3.predict(X_val))))
print(np.sqrt(mean_squared_error(y_val, modelXGB4.predict(X_val))))
print(np.sqrt(mean_squared_error(y_val, modelXGB5.predict(X_val))))
#모델4 선택

3501.426194616309
3912.538354425576
2980.8266047002044
3919.4580893900347


In [45]:
pred = modelXGB4.predict(X_test[COL_NUM+COL_CAT])
y_pred = pd.DataFrame({'id':X_test['id']
                       , 'pred' : pred})
y_pred.to_csv('수험번호.csv',index=False,encoding='utf-8')